# Панельная регрессия с Linearmodels. Подгонка регрессии через формулу

In [ ]:
import numpy as np
import pandas as pd

from linearmodels import PooledOLS     # Pool модель
from linearmodels import RandomEffects # RE-модель
from linearmodels import PanelOLS      # FE-модель

from linearmodels.panel import compare # несколько регрессий в одной таблице

## Загрузка и подготовка датасета

In [ ]:
df = pd.read_csv('Wages.csv')
df

In [ ]:
# зададим панельную структуру через мульти-индекс
# id - индивидуальный индекс, time - временной индекс
panel_df = df.set_index(['id', 'time'])
panel_df

## Панельные регрессии
Рассмотрим модель `lwage` на `ed, exp, exp**2, south, smsa, ind`  в трёх  вариантах:
- Pool
- Random effects
- Fixed effects

Т.к. формула спецификации одна и та же, то зададим её в виде строковой переменной (не забываем указать 1 в начале)


**Замечание** подгоним модели с настройками по умолчанию, т.е. с неробастной оценкой ковариационной матрицы

In [ ]:
spec = 'lwage~1+ed+exp+I(exp**2)+south+smsa+ind'

### Pool-модель

In [ ]:
mod_pl = PooledOLS.from_formula(formula=spec, data=panel_df)
res_pl = mod_pl.fit()

res_pl.summary

### RE-модель

In [ ]:
mod_re = RandomEffects.from_formula(formula=spec, data=panel_df)
res_re = mod_re.fit()

res_re.summary

### FE-модель
**Важно**:
- добавить к спецификации `+EntityEffects` и/или `+TimeEffects`
- указать `drop_absorbed=True` для контроля постоянных во времени переменных

In [ ]:
mod_fe = PanelOLS.from_formula(formula=spec+'+EntityEffects', data=panel_df, drop_absorbed=True)
res_fe = mod_fe.fit()

res_fe.summary

### Три модели в одной таблице
Модели оформляем в виде словаря

In [ ]:
compare(results={'Pool':res_pl, 'RE':res_re, 'FE':res_fe}, precision='std-errors', stars=True)

Сохраним талицу в файле

In [ ]:
panel_output = compare(results={'Pool':res_pl, 'RE':res_re, 'FE':res_fe}, precision='std-errors', stars=True)

with open('panel_output.txt', 'w') as f:
	f.write(panel_output.summary.as_text())